# Сравнение геометрий STEP с GNN

**Цель**: сравнить две 3D-модели в формате STEP, определить функциональные роли элементов и совместить модели.

In [115]:
import os
import sys
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from scipy.spatial.transform import Rotation as R

try:
    from OCC.Core import STEPControl, TopExp, TopAbs
    from OCC.Core.BRep import BRep_Tool
    from OCC.Core.gp import gp_Pnt
    import numpy as np
except ImportError:
    print("нет pythonocc-core")
    sys.exit(1)

In [116]:
from OCC.Core import TopoDS
print(dir(TopoDS))

['ClassNotWrappedError', 'CompSolid', 'Compound', 'Edge', 'Face', 'Handle_TopoDS_AlertAttribute_Create', 'Handle_TopoDS_AlertAttribute_DownCast', 'Handle_TopoDS_AlertAttribute_IsNull', 'Handle_TopoDS_AlertWithShape_Create', 'Handle_TopoDS_AlertWithShape_DownCast', 'Handle_TopoDS_AlertWithShape_IsNull', 'Handle_TopoDS_HShape_Create', 'Handle_TopoDS_HShape_DownCast', 'Handle_TopoDS_HShape_IsNull', 'Handle_TopoDS_TCompSolid_Create', 'Handle_TopoDS_TCompSolid_DownCast', 'Handle_TopoDS_TCompSolid_IsNull', 'Handle_TopoDS_TCompound_Create', 'Handle_TopoDS_TCompound_DownCast', 'Handle_TopoDS_TCompound_IsNull', 'Handle_TopoDS_TEdge_Create', 'Handle_TopoDS_TEdge_DownCast', 'Handle_TopoDS_TEdge_IsNull', 'Handle_TopoDS_TFace_Create', 'Handle_TopoDS_TFace_DownCast', 'Handle_TopoDS_TFace_IsNull', 'Handle_TopoDS_TShape_Create', 'Handle_TopoDS_TShape_DownCast', 'Handle_TopoDS_TShape_IsNull', 'Handle_TopoDS_TShell_Create', 'Handle_TopoDS_TShell_DownCast', 'Handle_TopoDS_TShell_IsNull', 'Handle_TopoDS_T

## 1 Парсинг STEP-файлов

In [117]:
def read_step_file(filename):
    """Загружает STEP-файл и возвращает TopoDS_Shape"""
    reader = STEPControl.STEPControl_Reader()
    reader.ReadFile(str(filename))
    reader.TransferRoots()
    return reader.OneShape()



In [118]:

os.makedirs("outputs/results", exist_ok=True)

#STEP_FILE_1 = "test.step"  
#STEP_FILE_2 = "test1.step" 

STEP_FILE_1 = "synthetic_dataset/raw/bracket_000002_trans_00.step"  
STEP_FILE_2 = "synthetic_dataset/raw/bracket_000002_trans_01.step" 

#STEP_FILE_1 = "synthetic_dataset/raw/bracket_000199_trans_00.step"  
#STEP_FILE_2 = "synthetic_dataset/raw/bracket_000199_trans_01.step" 

# Загрузка моделей
shape1 = read_step_file(STEP_FILE_1)
shape2 = read_step_file(STEP_FILE_2)

print(shape1, shape2)

<class 'TopoDS_Solid'> <class 'TopoDS_Solid'>


In [119]:
def extract_topology(shape):
    # Извлекает вершины и связи грань-вершина 
    vertices = []
    vertex_map = {}
    face_vertex_indices = []

    face_explorer = TopExp.TopExp_Explorer(shape, TopAbs.TopAbs_FACE)
    while face_explorer.More():
        face = face_explorer.Current()
        local_vertices = []

        edge_explorer = TopExp.TopExp_Explorer(face, TopAbs.TopAbs_EDGE)
        while edge_explorer.More():
            edge = edge_explorer.Current()
            
            # Вложенный эксплорер для вершин 
            vertex_explorer = TopExp.TopExp_Explorer(edge, TopAbs.TopAbs_VERTEX)
            while vertex_explorer.More():
                vertex = vertex_explorer.Current()
                p = BRep_Tool.Pnt(vertex)
                key = (round(p.X(), 6), round(p.Y(), 6), round(p.Z(), 6))
                if key not in vertex_map:
                    vertex_map[key] = len(vertices)
                    vertices.append(np.array([p.X(), p.Y(), p.Z()]))
                local_vertices.append(vertex_map[key])
                vertex_explorer.Next()
            
            edge_explorer.Next()

        if local_vertices:
            local_vertices = list(dict.fromkeys(local_vertices))
            face_vertex_indices.append(local_vertices)
        face_explorer.Next()

    return np.array(vertices), face_vertex_indices

## 2️ Построение графа

In [120]:
def normalize_coordinates(vertices):
    if len(vertices) == 0:
        return vertices
    center = vertices.mean(axis=0)
    scale = np.max(np.abs(vertices - center)) + 1e-8
    return (vertices - center) / scale

def build_graph(vertices, face_vertex_indices):
    n_vertices = len(vertices)
    n_faces = len(face_vertex_indices)

    vertices_norm = normalize_coordinates(vertices)
    node_coords = np.zeros((n_vertices + n_faces, 3))
    node_types = np.zeros(n_vertices + n_faces, dtype=int)

    # Вершины
    node_coords[:n_vertices] = vertices_norm
    node_types[:n_vertices] = 0

    # Грани
    for i, vtx_ids in enumerate(face_vertex_indices):
        if vtx_ids:
            center = vertices_norm[vtx_ids].mean(axis=0)
            node_coords[n_vertices + i] = center
            node_types[n_vertices + i] = 1

    # Рёбра
    edge_index = []
    for face_id, vtx_ids in enumerate(face_vertex_indices):
        for vtx_id in vtx_ids:
            edge_index.append([n_vertices + face_id, vtx_id])
            edge_index.append([vtx_id, n_vertices + face_id])

    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    x = torch.tensor(node_coords, dtype=torch.float)
    node_type = torch.tensor(node_types, dtype=torch.long)

    return Data(x=x, edge_index=edge_index, node_type=node_type)

## 3️ Архитектура GNN

In [100]:
import os
import json
import numpy as np
import torch
from torch_geometric.data import Dataset, Data
from OCC.Core.STEPControl import STEPControl_Reader

class ExistingSyntheticDataset(Dataset):
    """
    Датасет для обучения на УЖЕ СУЩЕСТВУЮЩИХ синтетических данных
    
    Структура директории (уже создана вашим скриптом):
        synthetic_dataset/
        ├── raw/          # STEP-файлы
        ├── annotations/  # JSON с разметкой ролей
        └── processed/    # Графы в формате .pt (будут созданы автоматически)
    """
    def __init__(self, root="synthetic_dataset", transform=None, pre_transform=None):
        # ⚠️ КРИТИЧНО: устанавливаем атрибуты ДО вызова super().__init__()
        self.role_mapping = {
            "decorative": 0,
            "functional": 1,
            "fastening": 2,
            "reference_plane": 3
        }
        super().__init__(root, transform, pre_transform)
    
    @property
    def raw_dir(self):
        return os.path.join(self.root, "raw")
    
    @property
    def ann_dir(self):
        return os.path.join(self.root, "annotations")
    
    @property
    def raw_file_names(self):
        # Берём ТОЛЬКО оригинальные модели (без _trans_)
        files = [f for f in os.listdir(self.raw_dir) if f.endswith(".step") and "_trans_" not in f]
        return sorted(files)
    
    @property
    def processed_file_names(self):
        return [f"data_{idx:06d}.pt" for idx in range(len(self.raw_file_names))]
    
    def process(self):
        """Преобразование существующих STEP-файлов в графы с разметкой"""
        print(f"🔄 Подготовка {len(self.raw_file_names)} существующих моделей для обучения...")
        
        for idx, step_file in enumerate(self.raw_file_names):
            # 1. Загрузка модели
            reader = STEPControl_Reader()
            reader.ReadFile(os.path.join(self.raw_dir, step_file))
            reader.TransferRoots()
            shape = reader.OneShape()
            
            # 2. Извлечение топологии
            vertices, face_vertex_indices = extract_topology(shape)
            
            # 3. Построение графа
            data = build_graph(vertices, face_vertex_indices)
            
            # 4. Загрузка разметки из JSON
            ann_file = os.path.splitext(step_file)[0] + ".json"
            ann_path = os.path.join(self.ann_dir, ann_file)
            
            if os.path.exists(ann_path):
                with open(ann_path, "r") as f:
                    annotations = json.load(f)
                
                n_vertices = len(vertices)
                n_faces = len(face_vertex_indices)
                node_roles = np.zeros(n_vertices + n_faces, dtype=np.int64)
                
                # Вершины → декоративные (роль 0)
                node_roles[:n_vertices] = self.role_mapping["decorative"]
                
                # Грани → размечаем по близости к аннотациям
                face_centers = data.x[n_vertices:].numpy()
                
                # Опорные плоскости (роль 3)
                for ref_plane in annotations.get("reference_planes", []):
                    ref_center = np.array(ref_plane["center"])
                    distances = np.linalg.norm(face_centers - ref_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 15.0:
                        node_roles[n_vertices + closest_idx] = self.role_mapping["reference_plane"]
                
                # Крепёжные элементы (роль 2)
                for fastening in annotations.get("fastening_elements", []):
                    fast_center = np.array(fastening["center"])
                    distances = np.linalg.norm(face_centers - fast_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 10.0:
                        node_roles[n_vertices + closest_idx] = self.role_mapping["fastening"]
                
                # Функциональные поверхности (роль 1)
                for func_surf in annotations.get("functional_surfaces", []):
                    func_center = np.array(func_surf["center"])
                    distances = np.linalg.norm(face_centers - func_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 10.0:
                        node_roles[n_vertices + closest_idx] = self.role_mapping["functional"]
                
                # Остальные грани → функциональные (роль 1)
                for i in range(n_faces):
                    if node_roles[n_vertices + i] == 0:
                        node_roles[n_vertices + i] = self.role_mapping["functional"]
                
                data.y = torch.tensor(node_roles, dtype=torch.long)
            else:
                # Без разметки — все грани функциональные
                n_vertices = len(vertices)
                n_faces = len(face_vertex_indices)
                node_roles = np.zeros(n_vertices + n_faces, dtype=np.int64)
                node_roles[:n_vertices] = self.role_mapping["decorative"]
                node_roles[n_vertices:] = self.role_mapping["functional"]
                data.y = torch.tensor(node_roles, dtype=torch.long)
            
            # 5. Сохранение графа БЕЗ весов (только данные)
            torch.save(data, os.path.join(self.processed_dir, f"data_{idx:06d}.pt"))
        
        print(f"✅ Подготовлено {len(self.raw_file_names)} моделей")
        print(f"   Графы сохранены в: {self.processed_dir}")
    
    def len(self):
        return len(self.processed_file_names)
    
    def get(self, idx):
        # ⚠️ КРИТИЧНО: weights_only=False для загрузки пользовательских объектов
        return torch.load(
            os.path.join(self.processed_dir, self.processed_file_names[idx]),
            weights_only=False  # Отключаем строгую безопасность (данные свои)
        )

проверка датасета


In [ ]:
# Проверяем, есть ли вообще роль 3 (опорные плоскости) в обучающих данных
test_data = dataset[0]
face_mask = (test_data.node_type == 1)
true_roles = test_data.y[face_mask]

print("📊 Анализ разметки первой модели:")
print(f"   Всего граней: {face_mask.sum().item()}")
print(f"   Распределение ролей (только грани):")
for role in range(4):
    count = (true_roles == role).sum().item()
    role_name = ["декоративная", "функциональная", "крепёжная", "опорная плоскость"][role]
    print(f"      Роль {role} ({role_name:20s}): {count:3d} шт.")

# Визуализация центроидов граней и аннотаций
print("\n📍 Центроиды граней из графа (первые 5):")
for i in range(min(5, face_mask.sum().item())):
    print(f"   Грань {i}: {test_data.x[len(test_data.x) - face_mask.sum().item() + i].numpy()}")

# Загрузка оригинальной аннотации
step_file = dataset.raw_file_names[0]
ann_file = os.path.splitext(step_file)[0] + ".json"
ann_path = os.path.join(dataset.ann_dir, ann_file)

if os.path.exists(ann_path):
    with open(ann_path, "r") as f:
        annotations = json.load(f)
    print(f"\n📄 Оригинальная аннотация из {ann_file}:")
    print(f"   Опорные плоскости: {len(annotations.get('reference_planes', []))} шт.")
    for i, rp in enumerate(annotations.get('reference_planes', [])):
        print(f"      {i+1}. Центр: {rp['center']}, Нормаль: {rp['normal']}, Площадь: {rp['area']}")

In [106]:
 class FixedSyntheticDataset(Dataset):
    """
    Исправленный датасет с правильным сопоставлением аннотаций
    """
    def __init__(self, root="synthetic_dataset", transform=None, pre_transform=None):
        self.role_mapping = {
            "decorative": 0,
            "functional": 1,
            "fastening": 2,
            "reference_plane": 3
        }
        super().__init__(root, transform, pre_transform)
    
    @property
    def raw_dir(self):
        return os.path.join(self.root, "raw")
    
    @property
    def ann_dir(self):
        return os.path.join(self.root, "annotations")
    
    @property
    def raw_file_names(self):
        files = [f for f in os.listdir(self.raw_dir) if f.endswith(".step") and "_trans_" not in f]
        return sorted(files)
    
    @property
    def processed_file_names(self):
        return [f"data_{idx:06d}.pt" for idx in range(len(self.raw_file_names))]
    
    def process(self):
        print(f"🔄 Подготовка {len(self.raw_file_names)} моделей...")
        
        for idx, step_file in enumerate(self.raw_file_names):
            # Загрузка модели
            reader = STEPControl_Reader()
            reader.ReadFile(os.path.join(self.raw_dir, step_file))
            reader.TransferRoots()
            shape = reader.OneShape()
            
            # Извлечение топологии БЕЗ нормализации для сопоставления аннотаций
            vertices, face_vertex_indices = extract_topology(shape)
            
            # Загрузка аннотаций
            ann_file = os.path.splitext(step_file)[0] + ".json"
            ann_path = os.path.join(self.ann_dir, ann_file)
            annotations = {}
            if os.path.exists(ann_path):
                with open(ann_path, "r") as f:
                    annotations = json.load(f)
            
            # Сопоставление аннотаций с гранями ДО нормализации
            n_vertices = len(vertices)
            n_faces = len(face_vertex_indices)
            node_roles = np.zeros(n_vertices + n_faces, dtype=np.int64)
            node_roles[:n_vertices] = self.role_mapping["decorative"]  # вершины → декоративные
            
            # Вычисляем центроиды граней в исходных координатах
            face_centers_raw = []
            for vtx_ids in face_vertex_indices:
                if vtx_ids:
                    center = vertices[vtx_ids].mean(axis=0)
                    face_centers_raw.append(center)
            face_centers_raw = np.array(face_centers_raw)
            
            # Сопоставление аннотаций
            assigned = np.zeros(n_faces, dtype=bool)
            
            # 1. Опорные плоскости (роль 3) — приоритет
            for ref_plane in annotations.get("reference_planes", []):
                ref_center = np.array(ref_plane["center"])
                if len(face_centers_raw) > 0:
                    distances = np.linalg.norm(face_centers_raw - ref_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 25.0:  # увеличенный порог 25 мм
                        node_roles[n_vertices + closest_idx] = self.role_mapping["reference_plane"]
                        assigned[closest_idx] = True
            
            # 2. Крепёжные элементы (роль 2)
            for fastening in annotations.get("fastening_elements", []):
                fast_center = np.array(fastening["center"])
                if len(face_centers_raw) > 0:
                    distances = np.linalg.norm(face_centers_raw - fast_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 15.0 and not assigned[closest_idx]:
                        node_roles[n_vertices + closest_idx] = self.role_mapping["fastening"]
                        assigned[closest_idx] = True
            
            # 3. Функциональные поверхности (роль 1)
            for func_surf in annotations.get("functional_surfaces", []):
                func_center = np.array(func_surf["center"])
                if len(face_centers_raw) > 0:
                    distances = np.linalg.norm(face_centers_raw - func_center, axis=1)
                    closest_idx = np.argmin(distances)
                    if distances[closest_idx] < 15.0 and not assigned[closest_idx]:
                        node_roles[n_vertices + closest_idx] = self.role_mapping["functional"]
                        assigned[closest_idx] = True
            
            # 4. Остальные грани → функциональные (роль 1)
            for i in range(n_faces):
                if not assigned[i]:
                    node_roles[n_vertices + i] = self.role_mapping["functional"]
            
            # Теперь строим граф С НОРМАЛИЗАЦИЕЙ (как в оригинальном коде)
            data = build_graph(vertices, face_vertex_indices)
            data.y = torch.tensor(node_roles, dtype=torch.long)
            
            # Сохранение
            torch.save(data, os.path.join(self.processed_dir, f"data_{idx:06d}.pt"))
        
        print(f"✅ Подготовлено {len(self.raw_file_names)} моделей")
    
    def len(self):
        return len(self.processed_file_names)
    
    def get(self, idx):
        return torch.load(
            os.path.join(self.processed_dir, self.processed_file_names[idx]),
            weights_only=False
            )

In [121]:
from torch_geometric.nn import GATv2Conv, global_mean_pool

class GNNModel(torch.nn.Module):
    def __init__(self, in_channels=3, hidden_dim=64, num_roles=4):
        super().__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_dim, heads=4, concat=True, dropout=0.2)
        self.conv2 = GATv2Conv(hidden_dim * 4, hidden_dim, heads=2, concat=False, dropout=0.2)
        self.role_classifier = torch.nn.Linear(hidden_dim, num_roles)
        self.graph_proj = torch.nn.Linear(hidden_dim, 64)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        roles = self.role_classifier(x)
        graph_emb = global_mean_pool(x, batch)
        graph_emb = self.graph_proj(graph_emb)
        return roles, graph_emb

##  Данные для обучения и обучение  


In [ ]:
# 1. Создание датасета из СУЩЕСТВУЮЩИХ данных
print("📦 Загрузка существующих синтетических данных...")
dataset = ExistingSyntheticDataset(root="synthetic_dataset")

# 2. Разделение на тренировочную/валидационную выборки
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(
    dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

from torch_geometric.loader import DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f"   Тренировка: {len(train_dataset)} моделей")
print(f"   Валидация:  {len(val_dataset)} моделей")

# 3. Инициализация модели
device = torch.device('cpu')  # или 'cuda' если есть GPU
model = GNNModel(in_channels=3, hidden_dim=64, num_roles=4).to(device)

# 4. Обучение
import torch.optim as optim

# Веса для балансировки классов (опорные плоскости редки)
class_weights = torch.tensor([0.5, 1.0, 1.5, 3.0], dtype=torch.float, device=device)
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

best_val_loss = float('inf')
print("\n🚀 Обучение на существующих данных (50 эпох)...\n")

for epoch in range(50):
    # Тренировка
    model.train()
    train_loss = 0
    for data in train_loader:
        data = data.to(device)
        batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)
        
        roles_pred, _ = model(data.x, data.edge_index, batch)
        
        # Потеря только для граней (игнорируем вершины)
        face_mask = (data.node_type == 1)
        if face_mask.sum() > 0:
            loss = criterion(roles_pred[face_mask], data.y[face_mask])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.num_graphs
    
    avg_train_loss = train_loss / len(train_loader.dataset)
    
    # Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)
            roles_pred, _ = model(data.x, data.edge_index, batch)
            
            face_mask = (data.node_type == 1)
            if face_mask.sum() > 0:
                loss = criterion(roles_pred[face_mask], data.y[face_mask])
                val_loss += loss.item() * data.num_graphs
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    
    # Сохранение лучшей модели
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "gnn_best.pth")
        status = "⭐"
    else:
        status = ""
    
    print(f"Эпоха {epoch+1:2d}/50 | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} {status}")

print(f"\n✅ Обучение завершено! Модель сохранена в: gnn_best.pth")

In [ ]:
# Создание исправленного датасета
print("📦 Переподготовка данных с исправленным сопоставлением аннотаций...")
dataset = FixedSyntheticDataset(root="synthetic_dataset")

# Проверка разметки
test_data = dataset[0]
face_mask = (test_data.node_type == 1)
true_roles = test_data.y[face_mask]
role3_count = (true_roles == 3).sum().item()
print(f"\n✅ Проверка разметки после исправления:")
print(f"   Опорных плоскостей (роль 3) в первой модели: {role3_count} из {face_mask.sum().item()} граней")
if role3_count == 0:
    print("⚠️  ВНИМАНИЕ: Даже после исправления нет опорных плоскостей!")
    print("   Возможно, проблема в синтетических данных — проверьте аннотации в JSON")
else:
    print("✅ Разметка корректна — начинаем обучение")

# Разделение и загрузчики
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Модель
device = torch.device('cpu')
model = GNNModel(in_channels=3, hidden_dim=64, num_roles=4).to(device)

# Усиленные веса для роли 3 (опорные плоскости)
class_weights = torch.tensor([0.2, 0.3, 1.0, 10.0], dtype=torch.float, device=device)  # роль 3 имеет вес 10!
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Обучение с ранней остановкой
best_val_loss = float('inf')
patience = 0
max_patience = 15

print("\n🚀 Обучение с усиленным фокусом на опорные плоскости (роль 3)...\n")

for epoch in range(150):  # увеличено до 150 эпох
    # Тренировка
    model.train()
    train_loss = 0
    train_correct_role3 = 0
    train_total_role3 = 0
    
    for data in train_loader:
        data = data.to(device)
        batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)
        
        roles_pred, _ = model(data.x, data.edge_index, batch)
        
        face_mask = (data.node_type == 1)
        if face_mask.sum() > 0:
            loss = criterion(roles_pred[face_mask], data.y[face_mask])
            
            # Счётчик точности для роли 3
            preds = roles_pred[face_mask].argmax(dim=1)
            role3_mask = (data.y[face_mask] == 3)
            if role3_mask.sum() > 0:
                train_correct_role3 += (preds[role3_mask] == 3).sum().item()
                train_total_role3 += role3_mask.sum().item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.num_graphs
    
    avg_train_loss = train_loss / len(train_loader.dataset)
    role3_acc = train_correct_role3 / train_total_role3 if train_total_role3 > 0 else 0
    
    # Валидация
    model.eval()
    val_loss = 0
    val_correct_role3 = 0
    val_total_role3 = 0
    
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            batch = torch.zeros(data.x.size(0), dtype=torch.long, device=device)
            roles_pred, _ = model(data.x, data.edge_index, batch)
            
            face_mask = (data.node_type == 1)
            if face_mask.sum() > 0:
                loss = criterion(roles_pred[face_mask], data.y[face_mask])
                val_loss += loss.item() * data.num_graphs
                
                preds = roles_pred[face_mask].argmax(dim=1)
                role3_mask = (data.y[face_mask] == 3)
                if role3_mask.sum() > 0:
                    val_correct_role3 += (preds[role3_mask] == 3).sum().item()
                    val_total_role3 += role3_mask.sum().item()
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    val_role3_acc = val_correct_role3 / val_total_role3 if val_total_role3 > 0 else 0
    
    # Сохранение лучшей модели
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "gnn_best.pth")
        patience = 0
        status = "⭐"
    else:
        patience += 1
        status = ""
    
    print(f"Эпоха {epoch+1:3d}/150 | "
          f"Loss: {avg_train_loss:.4f} → {avg_val_loss:.4f} | "
          f"RefAcc: {role3_acc:.1%} → {val_role3_acc:.1%} {status}")
    
    if patience >= max_patience:
        print(f"\n⏹️  Ранняя остановка на эпохе {epoch+1}")
        break

print(f"\n✅ Обучение завершено! Лучшая модель сохранена в: gnn_best.pth")

# тест

In [122]:
# Загрузка и тест
model = GNNModel(in_channels=3, hidden_dim=64, num_roles=4)
model.load_state_dict(torch.load("gnn_best.pth", map_location='cpu'))
model.eval()

test_data = dataset[0]
with torch.no_grad():
    batch = torch.zeros(test_data.x.size(0), dtype=torch.long)
    roles_pred, _ = model(test_data.x, test_data.edge_index, batch)
    pred_classes = roles_pred.argmax(dim=1)

face_mask = (test_data.node_type == 1)
true_roles = test_data.y[face_mask]
pred_roles = pred_classes[face_mask]

print("\n✅ Финальная проверка качества обучения:")
print(f"   Всего граней: {face_mask.sum().item()}")
print(f"   Истинных опорных плоскостей (роль 3): {(true_roles == 3).sum().item()}")
print(f"   Предсказанных опорных плоскостей (роль 3): {(pred_roles == 3).sum().item()}")

# Подробная таблица совпадений
print("\n📋 Детальное сравнение (первые 10 граней):")
print("Грань | Истинная роль | Предсказанная роль | Совпадение")
print("-" * 55)
for i in range(min(10, len(true_roles))):
    match = "✅" if true_roles[i] == pred_roles[i] else "❌"
    true_name = ["декор", "функц", "крепёж", "опорн"][true_roles[i]]
    pred_name = ["декор", "функц", "крепёж", "опорн"][pred_roles[i]]
    print(f" {i:3d}  | {true_name:12s} | {pred_name:16s} | {match}")

# Тест на реальных моделях
print("\n🔍 Тест на ваших реальных моделях (если есть):")
v1, fv1 = extract_topology(shape1)
g1 = build_graph(v1, fv1)

with torch.no_grad():
    batch1 = torch.zeros(g1.x.size(0), dtype=torch.long)
    roles1, _ = model(g1.x, g1.edge_index, batch1)
    ref1 = find_reference_planes(roles1.softmax(dim=1).numpy(), g1.node_type.numpy(), g1.x[:, :3].numpy())

print(f"   Найдено опорных плоскостей в реальной модели: {len(ref1)}")
if len(ref1) >= 3:
    print("✅ Модель готова к автоматическому совмещению!")
else:
    print("⚠️  Недостаточно опорных плоскостей — требуется дообучение на более сложных данных")


✅ Финальная проверка качества обучения:
   Всего граней: 14
   Истинных опорных плоскостей (роль 3): 0
   Предсказанных опорных плоскостей (роль 3): 0

📋 Детальное сравнение (первые 10 граней):
Грань | Истинная роль | Предсказанная роль | Совпадение
-------------------------------------------------------
   0  | функц        | функц            | ✅
   1  | функц        | функц            | ✅
   2  | функц        | функц            | ✅
   3  | функц        | функц            | ✅
   4  | функц        | функц            | ✅
   5  | функц        | функц            | ✅
   6  | функц        | функц            | ✅
   7  | функц        | функц            | ✅
   8  | функц        | функц            | ✅
   9  | функц        | функц            | ✅

🔍 Тест на ваших реальных моделях (если есть):
   Найдено опорных плоскостей в реальной модели: 3
✅ Модель готова к автоматическому совмещению!


In [123]:
# Загрузка обученной модели
model = GNNModel(in_channels=3, hidden_dim=64, num_roles=4)
model.load_state_dict(torch.load("gnn_best.pth", map_location='cpu'))
model.eval()

# Обработка ваших реальных моделей (из ноутбука)
v1, fv1 = extract_topology(shape1)
v2, fv2 = extract_topology(shape2)

g1 = build_graph(v1, fv1)
g2 = build_graph(v2, fv2)

# Предсказание ролей
with torch.no_grad():
    batch1 = torch.zeros(g1.x.size(0), dtype=torch.long)
    batch2 = torch.zeros(g2.x.size(0), dtype=torch.long)
    roles1, _ = model(g1.x, g1.edge_index, batch1)
    roles2, _ = model(g2.x, g2.edge_index, batch2)

# Поиск опорных плоскостей (роль 3)
ref1 = find_reference_planes(roles1.softmax(dim=1).numpy(), g1.node_type.numpy(), g1.x[:, :3].numpy())
ref2 = find_reference_planes(roles2.softmax(dim=1).numpy(), g2.node_type.numpy(), g2.x[:, :3].numpy())

print(f"\n🔍 Результаты предсказания обученной моделью:")
print(f"   Опорных плоскостей в модели 1: {len(ref1)}")
print(f"   Опорных плоскостей в модели 2: {len(ref2)}")

if len(ref1) >= 3 and len(ref2) >= 3:
    print("✅ Модель успешно находит опорные плоскости!")
    print("   Теперь можно выполнять автоматическое совмещение...")
    
    # Совмещение (ваша существующая функция)
    R_mat, t_vec = align_with_umeyama(ref1[:3], ref2[:3])
    
    # Применение преобразования (ваш существующий код)
    from OCC.Core.gp import gp_Trsf
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
    
    trsf = gp_Trsf()
    trsf.SetValues(
        float(R_mat[0,0]), float(R_mat[0,1]), float(R_mat[0,2]), float(t_vec[0]),
        float(R_mat[1,0]), float(R_mat[1,1]), float(R_mat[1,2]), float(t_vec[1]),
        float(R_mat[2,0]), float(R_mat[2,1]), float(R_mat[2,2]), float(t_vec[2])
    )
    
    transform = BRepBuilderAPI_Transform(shape2, trsf)
    shape2_aligned = transform.Shape()
    
    # Сохранение результата
    from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
    os.makedirs("outputs/results", exist_ok=True)
    writer = STEPControl_Writer()
    writer.Transfer(shape2_aligned, STEPControl_AsIs)
    writer.Write("outputs/results/aligned_with_gnn.step")
    
    print("✅ Совмещённая модель сохранена: outputs/results/aligned_with_gnn.step")
else:
    print("⚠️  Модель не находит достаточно опорных плоскостей")
    print("   Возможные причины:")
    print("   • Модель обучена на простых синтетических данных")
    print("   • Реальные модели сложнее синтетических")
    print("   • Нужно дообучить модель на 500+ моделях или добавить аугментации")


🔍 Результаты предсказания обученной моделью:
   Опорных плоскостей в модели 1: 3
   Опорных плоскостей в модели 2: 3
✅ Модель успешно находит опорные плоскости!
   Теперь можно выполнять автоматическое совмещение...
✅ Совмещённая модель сохранена: outputs/results/aligned_with_gnn.step

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 2                      ******
** WorkSession : Sending all data
 Step File Name : outputs/results/aligned_with_gnn.step(882 ents)  Write  Done


In [ ]:
# Загрузка обученной модели
model = GNNModel(in_channels=3, hidden_dim=64, num_roles=4)
model.load_state_dict(torch.load("gnn_best.pth", map_location='cpu'))
model.eval()

# Тест на случайной модели из датасета
test_data = dataset[0]
with torch.no_grad():
    batch = torch.zeros(test_data.x.size(0), dtype=torch.long)
    roles_pred, _ = model(test_data.x, test_data.edge_index, batch)
    pred_classes = roles_pred.argmax(dim=1)

# Подсчёт количества опорных плоскостей (роль 3) среди граней
face_mask = (test_data.node_type == 1)
ref_planes_found = (pred_classes[face_mask] == 3).sum().item()
total_faces = face_mask.sum().item()

print(f"\n🔍 Тест на первой модели из датасета:")
print(f"   Всего граней: {total_faces}")
print(f"   Найдено опорных плоскостей (роль 3): {ref_planes_found}")
print(f"   Процент опорных плоскостей: {ref_planes_found / total_faces * 100:.1f}%")
print(f"   {'✅ Достаточно для совмещения (≥3)' if ref_planes_found >= 3 else '⚠️ Недостаточно для совмещения'}")

## 4️ Совмещение моделей

In [110]:
def find_reference_planes(role_probs, node_types, coords, top_k=3):
    face_mask = (node_types == 1)
    if not np.any(face_mask):
        return np.array([])
    
    ref_scores = role_probs[face_mask, 3]  # индекс 3 = reference_plane
    top_indices = np.argsort(-ref_scores)[:top_k]
    face_indices = np.where(face_mask)[0][top_indices]
    return coords[face_indices]

def align_with_umeyama(src_pts, tgt_pts):
    assert src_pts.shape == tgt_pts.shape and src_pts.shape[0] >= 3
    
    src_mean = src_pts.mean(axis=0)
    tgt_mean = tgt_pts.mean(axis=0)
    src_c = src_pts - src_mean
    tgt_c = tgt_pts - tgt_mean
    
    H = src_c.T @ tgt_c
    U, S, Vt = np.linalg.svd(H)
    R_opt = Vt.T @ U.T
    
    if np.linalg.det(R_opt) < 0:
        Vt[-1, :] *= -1
        R_opt = Vt.T @ U.T
        
    t_opt = tgt_mean - R_opt @ src_mean
    return R_opt, t_opt

## 5️ Загрузка данных и запуск 


In [124]:

# Извлечение топологии
v1, fv1 = extract_topology(shape1)
v2, fv2 = extract_topology(shape2)

# Построение графов
g1 = build_graph(v1, fv1)
g2 = build_graph(v2, fv2)

print(f"Модель 1: {g1.x.size(0)} узлов")
print(f"Модель 2: {g2.x.size(0)} узлов")

Модель 1: 34 узлов
Модель 2: 34 узлов


In [125]:
# Инициализация модели
model = GNNModel()

# Загрузка весов 
try:
    model.load_state_dict(torch.load("gnn_best.pth", map_location='cpu'))
    print("✅ Модель загружена")
except FileNotFoundError:
    print("⚠️ Модель не найдена. Используем случайные веса.")

model.eval()

# Получение эмбеддингов
with torch.no_grad():
    batch1 = torch.zeros(g1.x.size(0), dtype=torch.long)
    batch2 = torch.zeros(g2.x.size(0), dtype=torch.long)
    roles1, emb1 = model(g1.x, g1.edge_index, batch1)
    roles2, emb2 = model(g2.x, g2.edge_index, batch2)

# Метрика схожести
similarity = float(torch.cosine_similarity(emb1, emb2).item())
print(f"\n🎯 Метрика схожести: {similarity:.3f}")

✅ Модель загружена

🎯 Метрика схожести: 1.000


In [126]:
# Совмещение
ref1 = find_reference_planes(roles1.softmax(dim=1).numpy(), g1.node_type.numpy(), g1.x[:, :3].numpy())
ref2 = find_reference_planes(roles2.softmax(dim=1).numpy(), g2.node_type.numpy(), g2.x[:, :3].numpy())

if len(ref1) >= 3 and len(ref2) >= 3:
    R_mat, t_vec = align_with_umeyama(ref1[:3], ref2[:3])
    print("совмещение выполнено")
else:
    R_mat, t_vec = np.eye(3), np.zeros(3)
    print("едостаточно опорных плоскостей для совмещения")

print(f"Матрица поворота:\n{R_mat}")
print(f"Вектор сдвига: {t_vec}")

совмещение выполнено
Матрица поворота:
[[-0.46977624 -0.20261793  0.85921836]
 [ 0.611248   -0.7769023   0.15099236]
 [ 0.636935    0.5961281   0.48882005]]
Вектор сдвига: [ 0.65383744 -0.5989876  -0.61852497]


## 6️ Экспорт совмещённой модели

In [127]:
# Применение преобразования
from OCC.Core.gp import gp_Trsf
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
import os

# Создаём директорию для результатов
os.makedirs("outputs/results", exist_ok=True)

# Создаём преобразование (ТОЛЬКО 12 аргументов: 3×4 матрица)
trsf = gp_Trsf()
trsf.SetValues(
    float(R_mat[0,0]), float(R_mat[0,1]), float(R_mat[0,2]), float(t_vec[0]),
    float(R_mat[1,0]), float(R_mat[1,1]), float(R_mat[1,2]), float(t_vec[1]),
    float(R_mat[2,0]), float(R_mat[2,1]), float(R_mat[2,2]), float(t_vec[2])
)

# Применяем преобразование
transform = BRepBuilderAPI_Transform(shape2, trsf)
shape2_aligned = transform.Shape()

# Сохранение
writer = STEPControl_Writer()
writer.Transfer(shape2_aligned, STEPControl_AsIs)
status = writer.Write("outputs/results/aligned_model.step")

if status == 1:  # IFSelect_RetDone
    print("\n✅ Совмещённая модель сохранена в outputs/results/aligned_model.step")
else:
    print(f"\n❌ Ошибка при сохранении STEP-файла (код статуса: {status})")


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 2                      ******

✅ Совмещённая модель сохранена в outputs/results/aligned_model.step
** WorkSession : Sending all data
 Step File Name : outputs/results/aligned_model.step(882 ents)  Write  Done


In [128]:
def check_intersection(shape1, shape2, tolerance=1e-3):
    """Проверяет, пересекаются ли модели после совмещения"""
    from OCC.Core.BRepExtrema import BRepExtrema_DistShapeShape
    from OCC.Core.gp import gp_Pnt
    
    # Проверяем расстояние между поверхностями
    dist_checker = BRepExtrema_DistShapeShape(shape1, shape2)
    dist_checker.Perform()
    
    if dist_checker.IsDone():
        min_dist = dist_checker.Value()
        print(f"🔍 Минимальное расстояние между моделями: {min_dist:.6f} мм")
        if min_dist < tolerance:
            print("✅ Модели пересекаются — можно выполнять булевы операции")
        else:
            print("⚠️ Модели НЕ пересекаются — увеличьте сдвиг/поворот")
        return min_dist
    else:
        print("❌ Не удалось проверить пересечение")
        return None

In [139]:
# Проверка пересечения
check_intersection(shape1, shape2_aligned, tolerance=1e-2)

🔍 Минимальное расстояние между моделями: 20.950899 мм
⚠️ Модели НЕ пересекаются — увеличьте сдвиг/поворот


20.950898684766774

In [129]:
# Булевы операции для сравнения моделей
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut, BRepAlgoAPI_Fuse
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
import os

def get_solid(shape):
    """Извлекает первое твердое тело (Solid) из формы"""
    explorer = TopExp_Explorer(shape, TopAbs_SOLID)
    if explorer.More():
        return explorer.Current()  # В вашей версии приведение не требуется
    else:
        raise ValueError("Форма не содержит твердых тел (Solids). Убедитесь, что файл содержит объемную геометрию.")

def boolean_symmetric_difference(shape1, shape2, tolerance=1e-5):
    """
    Вычисляет симметрическую разность: (A - B) ∪ (B - A)
    Показывает ВСЕ различия между моделями:
      - Красный: есть в модели 1, но нет в модели 2
      - Синий: есть в модели 2, но нет в модели 1
    """
    solid1 = get_solid(shape1)
    solid2 = get_solid(shape2)
    
    try:
        # Часть 1: что есть в модели 1, но отсутствует в модели 2
        cut1 = BRepAlgoAPI_Cut(solid1, solid2)
        cut1.SetFuzzyValue(tolerance)  # Улучшает стабильность для близких поверхностей
        if not cut1.IsDone():
            print("⚠️ Вычитание (модель1 - модель2) не выполнено")
            diff1 = None
        else:
            diff1 = cut1.Shape()
        
        # Часть 2: что есть в модели 2, но отсутствует в модели 1
        cut2 = BRepAlgoAPI_Cut(solid2, solid1)
        cut2.SetFuzzyValue(tolerance)
        if not cut2.IsDone():
            print("⚠️ Вычитание (модель2 - модель1) не выполнено")
            diff2 = None
        else:
            diff2 = cut2.Shape()
        
        # Объединяем различия
        if diff1 is None and diff2 is None:
            raise RuntimeError("Обе операции вычитания завершились неудачей")
        elif diff1 is None:
            return diff2
        elif diff2 is None:
            return diff1
        else:
            fuse = BRepAlgoAPI_Fuse(diff1, diff2)
            fuse.SetFuzzyValue(tolerance)
            if fuse.IsDone():
                return fuse.Shape()
            else:
                # Если объединение не удалось — возвращаем обе части отдельно
                return (diff1, diff2)
                
    except Exception as e:
        raise RuntimeError(f"Ошибка булевой операции: {str(e)}\n"
                         f"Совет: убедитесь, что модели пересекаются после совмещения")

def save_step(shape, filename):
    """Сохраняет форму в STEP-файл"""
    os.makedirs(os.path.dirname(filename) or ".", exist_ok=True)
    writer = STEPControl_Writer()
    writer.Transfer(shape, STEPControl_AsIs)
    status = writer.Write(filename)
    return status == 1  # IFSelect_RetDone = 1

In [130]:
# === Булевы операции для визуализации различий ===
print("\n🔍 Анализ различий между моделями...")

try:
    # Симметрическая разность — показывает ВСЕ отличия
    diff = boolean_symmetric_difference(shape1, shape2_aligned)
    
    if isinstance(diff, tuple):
        # Если объединение не удалось — сохраняем обе части отдельно
        save_step(diff[0], "outputs/results/differences_model1_only.step")
        save_step(diff[1], "outputs/results/differences_model2_only.step")
        print("✅ Различия сохранены в двух файлах:")
        print("   - differences_model1_only.step : элементы только в первой модели")
        print("   - differences_model2_only.step : элементы только во второй модели")
    else:
        save_step(diff, "outputs/results/differences_combined.step")
        print("✅ Все различия сохранены в: outputs/results/differences_combined.step")
        print("💡 Совет: откройте файл в FreeCAD и примените цветовую раскраску:")
        print("   - Красный: уникальные элементы первой модели")
        print("   - Синий: уникальные элементы второй модели")
        
except Exception as e:
    print(f"❌ Не удалось выполнить булевы операции: {e}")
    print("   Причины могут быть:")
    print("   • Модели не пересекаются после совмещения")
    print("   • Геометрия содержит дефекты (неправильные нормали, самопересечения)")
    print("   • Требуется увеличить параметр tolerance в функции")


🔍 Анализ различий между моделями...

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
✅ Все различия сохранены в: outputs/results/differences_combined.step
💡 Совет: откройте файл в FreeCAD и примените цветовую раскраску:
   - Красный: уникальные элементы первой модели
   - Синий: уникальные элементы второй модели
** WorkSession : Sending all data
 Step File Name : outputs/results/differences_combined.step(2982 ents)  Write  Done


# сделать 
- Для обучения модели создайте синтетические данные (пример в README)
-  Результат можно открыть в FreeCAD или любой CAD-системе

In [131]:
def extract_large_planes(shape, min_area=100.0, max_planes=5):
    """
    Извлечение крупных плоских граней без нейросети (гарантированно работает с pythonocc-core 7.9.0)
    
    Параметры:
    ----------
    shape : TopoDS_Shape
        CAD-модель в формате STEP
    min_area : float
        Минимальная площадь грани для включения (мм²)
    max_planes : int
        Максимальное число возвращаемых плоскостей
    
    Возвращает:
    -----------
    list[dict] : Список плоскостей с ключами:
        - 'area': площадь грани (мм²)
        - 'center': центроид грани (np.array[3])
        - 'normal': нормаль к плоскости (np.array[3])
    """
    from OCC.Core.TopExp import TopExp_Explorer
    from OCC.Core.TopAbs import TopAbs_FACE
    from OCC.Core.BRep import BRep_Tool
    from OCC.Core.BRepGProp import brepgprop_SurfaceProperties  # Старый синтаксис — работает в 7.9.0
    from OCC.Core.GProp import GProp_GProps
    from OCC.Core.Geom import Geom_Plane
    from OCC.Core.gp import gp_Pnt
    import numpy as np
    
    planes = []
    explorer = TopExp_Explorer(shape, TopAbs_FACE)
    
    while explorer.More():
        face = explorer.Current()  # Работаем напрямую с TopoDS_Shape (без приведения типов!)
        
        # Проверка: плоская ли грань?
        surface = BRep_Tool.Surface(face)
        if not surface.IsKind("Geom_Plane"):
            explorer.Next()
            continue
        
        # Вычисление площади грани (старый синтаксис — работает без предупреждений в 7.9.0)
        props = GProp_GProps()
        brepgprop_SurfaceProperties(face, props)  # ← работает в 7.9.0 из conda-forge
        area = props.Mass()
        
        if area < min_area:
            explorer.Next()
            continue
        
        # Центроид грани
        cog = props.CentreOfMass()
        
        # Нормаль к плоскости (без IsNull — проверяем через try/except)
        try:
            plane = Geom_Plane.DownCast(surface)
            # В 7.9.0 DownCast возвращает объект, даже если каст не удался
            # Проверяем через доступ к методам
            normal = plane.Axis().Direction()
        except:
            explorer.Next()
            continue
        
        planes.append({
            'area': area,
            'center': np.array([cog.X(), cog.Y(), cog.Z()]),
            'normal': np.array([normal.X(), normal.Y(), normal.Z()])
        })
        
        explorer.Next()
    
    # Сортировка по площади (крупнейшие первыми)
    planes.sort(key=lambda x: x['area'], reverse=True)
    return planes[:max_planes]


def evaluate_alignment(shape1, shape2_aligned, n_max_vertices=2000):
    """
    Оценка качества совмещения через вершины (без сложной выборки точек)
    Использует вашу рабочую функцию extract_topology
    
    Параметры:
    ----------
    shape1 : TopoDS_Shape
        Первая модель (цель)
    shape2_aligned : TopoDS_Shape
        Вторая модель после совмещения
    n_max_vertices : int
        Максимальное число вершин для анализа
    
    Возвращает:
    -----------
    dict : Метрики качества совмещения
    """
    from OCC.Core.BRepExtrema import BRepExtrema_DistShapeShape
    import numpy as np
    from scipy.spatial import cKDTree
    
    # Шаг 1: Минимальное расстояние Хаусдорфа (быстрая оценка)
    dist_checker = BRepExtrema_DistShapeShape()
    dist_checker.LoadS1(shape1)
    dist_checker.LoadS2(shape2_aligned)
    dist_checker.Perform()
    
    if not dist_checker.IsDone():
        return {'success': False, 'error': 'Расчёт расстояния не удался'}
    
    min_dist = dist_checker.Value()
    
    # Шаг 2: Извлечение вершин через ВАШУ рабочую функцию extract_topology
    vertices1, _ = extract_topology(shape1)
    vertices2, _ = extract_topology(shape2_aligned)
    
    # Ограничение числа вершин для скорости
    if len(vertices1) > n_max_vertices:
        indices = np.random.choice(len(vertices1), n_max_vertices, replace=False)
        vertices1 = vertices1[indices]
    if len(vertices2) > n_max_vertices:
        indices = np.random.choice(len(vertices2), n_max_vertices, replace=False)
        vertices2 = vertices2[indices]
    
    if len(vertices1) == 0 or len(vertices2) == 0:
        return {'success': False, 'error': 'Не удалось извлечь вершины'}
    
    # Шаг 3: Расчёт расстояний до ближайших точек
    tree2 = cKDTree(vertices2)
    dists12, _ = tree2.query(vertices1, k=1)
    
    tree1 = cKDTree(vertices1)
    dists21, _ = tree1.query(vertices2, k=1)
    
    # Метрики качества
    hausdorff_sym = max(dists12.max(), dists21.max())
    mean_dist = (dists12.mean() + dists21.mean()) / 2.0
    
    return {
        'success': True,
        'hausdorff_min': min_dist,
        'hausdorff_symmetric': hausdorff_sym,
        'mean_distance': mean_dist,
        'max_distance': max(dists12.max(), dists21.max()),
        'rms_distance': np.sqrt((dists12**2).mean()),
        'inlier_ratio_0.1mm': (dists12 < 0.1).mean(),
        'inlier_ratio_0.5mm': (dists12 < 0.5).mean(),
        'inlier_ratio_1.0mm': (dists12 < 1.0).mean(),
        'sample_count': len(vertices1)
    }

In [133]:
# 1. Извлечение опорных плоскостей без GNN
planes1 = extract_large_planes(shape1, min_area=50.0, max_planes=5)
planes2 = extract_large_planes(shape2, min_area=50.0, max_planes=5)

print(f"Найдено плоскостей в модели 1: {len(planes1)}")
print(f"Найдено плоскостей в модели 2: {len(planes2)}")

# Вывод информации о найденных плоскостях
for i, p in enumerate(planes1[:min(3, len(planes1))]):
    print(f"  Плоскость {i+1}: площадь={p['area']:.1f} мм², центр={p['center']}")

# 2. Совмещение по 3 крупнейшим плоскостям
if len(planes1) >= 3 and len(planes2) >= 3:
    src_pts = np.array([p['center'] for p in planes1[:3]])
    tgt_pts = np.array([p['center'] for p in planes2[:3]])
    
    # Используем вашу уже имеющуюся функцию align_with_umeyama
    R_mat, t_vec = align_with_umeyama(src_pts, tgt_pts)
    
    print("\n✅ Совмещение по опорным плоскостям выполнено")
    print(f"Матрица поворота:\n{R_mat}")
    print(f"Вектор сдвига: {t_vec}")
    
    # 3. Применение преобразования
    from OCC.Core.gp import gp_Trsf
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform
    from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
    import os
    
    trsf = gp_Trsf()
    trsf.SetValues(
        float(R_mat[0,0]), float(R_mat[0,1]), float(R_mat[0,2]), float(t_vec[0]),
        float(R_mat[1,0]), float(R_mat[1,1]), float(R_mat[1,2]), float(t_vec[1]),
        float(R_mat[2,0]), float(R_mat[2,1]), float(R_mat[2,2]), float(t_vec[2])
    )
    
    transform = BRepBuilderAPI_Transform(shape2, trsf)
    shape2_aligned = transform.Shape()
    
    # 4. Оценка качества совмещения
    print("\n📊 Оценка качества совмещения...")
    metrics = evaluate_alignment(shape1, shape2_aligned, n_max_vertices=2000)
    
    if metrics['success']:
        print(f"   Средняя ошибка:      {metrics['mean_distance']:.4f} мм")
        print(f"   Макс. ошибка:        {metrics['max_distance']:.4f} мм")
        print(f"   RMS ошибка:          {metrics['rms_distance']:.4f} мм")
        print(f"   Точек < 0.1 мм:      {metrics['inlier_ratio_0.1mm']*100:.1f}%")
        print(f"   Точек < 1.0 мм:      {metrics['inlier_ratio_1.0mm']*100:.1f}%")
        
        # 5. Сохранение результата
        os.makedirs("outputs/results", exist_ok=True)
        writer = STEPControl_Writer()
        writer.Transfer(shape2_aligned, STEPControl_AsIs)
        writer.Write("outputs/results/aligned_by_planes.step")
        print("\n✅ Совмещённая модель сохранена: outputs/results/aligned_by_planes.step")
    else:
        print(f"❌ Ошибка оценки: {metrics['error']}")
else:
    print("⚠️ Недостаточно плоских граней для совмещения (требуется минимум 3)")
    print("   Советы:")
    print("   • Уменьшите min_area (попробуйте 10.0 или даже 1.0)")
    print("   • Проверьте модели в FreeCAD — есть ли в них плоские поверхности?")
    print("   • Для моделей без плоскостей используйте совмещение по центроидам (см. ниже)")

Найдено плоскостей в модели 1: 5
Найдено плоскостей в модели 2: 5
  Плоскость 1: площадь=4842.9 мм², центр=[32.63397457 28.74798519 22.61988315]
  Плоскость 2: площадь=2171.5 мм², центр=[46.08245502 33.49638683 48.26793349]
  Плоскость 3: площадь=2171.5 мм², центр=[ 5.63086121 25.47563883 11.60375637]

✅ Совмещение по опорным плоскостям выполнено
Матрица поворота:
[[-0.79367806 -0.42209879 -0.43807277]
 [ 0.01031397 -0.7293473   0.68406589]
 [-0.60825058  0.53840982  0.58322046]]
Вектор сдвига: [ 92.93298127  12.81374282 -20.57567155]

📊 Оценка качества совмещения...
   Средняя ошибка:      61.3314 мм
   Макс. ошибка:        87.5443 мм
   RMS ошибка:          61.6722 мм
   Точек < 0.1 мм:      0.0%
   Точек < 1.0 мм:      0.0%

✅ Совмещённая модель сохранена: outputs/results/aligned_by_planes.step


/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/3267780437.py:44: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)  # ← работает в 7.9.0 из conda-forge
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/3267780437.py:44: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(face, props)  # ← работает в 7.9.0 из conda-forge
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/3267780437.py:44: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 2                      ******
** WorkSession : Sending all data
 Step File Name : outputs/results/aligned_by_planes.step(890 ents)  Write  Done


In [134]:
def ensure_solid(shape):
    """
    Преобразует форму в твёрдое тело (Solid), если возможно.
    Возвращает первый найденный Solid или исходную форму.
    """
    from OCC.Core.TopExp import TopExp_Explorer
    from OCC.Core.TopAbs import TopAbs_SOLID
    from OCC.Core.TopoDS import topods_Solid
    
    explorer = TopExp_Explorer(shape, TopAbs_SOLID)
    if explorer.More():
        try:
            return topods_Solid(explorer.Current())
        except:
            pass
    return shape  # Возвращаем как есть (может быть Shell)

In [135]:
def boolean_operations(shape1, shape2_aligned, tolerance=1e-5):
    """
    Выполняет булевы операции для сравнения совмещённых моделей
    
    Возвращает словарь с результатами:
        - 'fuse': объединение (A ∪ B)
        - 'common': пересечение (A ∩ B)
        - 'diff1': разность (A \ B) — что есть в модели 1, но нет во 2
        - 'diff2': разность (B \ A) — что есть в модели 2, но нет в 1
        - 'symdiff': симметрическая разность — ВСЕ различия
    """
    from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse, BRepAlgoAPI_Common, BRepAlgoAPI_Cut
    from OCC.Core.TopoDS import TopoDS_Solid
    import numpy as np
    
    # Преобразуем в твёрдые тела
    solid1 = ensure_solid(shape1)
    solid2 = ensure_solid(shape2_aligned)
    
    results = {}
    errors = []
    
    # 1. Объединение (A ∪ B)
    try:
        fuse = BRepAlgoAPI_Fuse(solid1, solid2)
        fuse.SetFuzzyValue(tolerance)  # Улучшает стабильность для близких поверхностей
        if fuse.IsDone():
            results['fuse'] = fuse.Shape()
        else:
            errors.append("fuse")
    except Exception as e:
        errors.append(f"fuse: {str(e)}")
    
    # 2. Пересечение (A ∩ B)
    try:
        common = BRepAlgoAPI_Common(solid1, solid2)
        common.SetFuzzyValue(tolerance)
        if common.IsDone():
            results['common'] = common.Shape()
        else:
            errors.append("common")
    except Exception as e:
        errors.append(f"common: {str(e)}")
    
    # 3. Разность (A \ B) — уникальные части модели 1
    try:
        cut1 = BRepAlgoAPI_Cut(solid1, solid2)
        cut1.SetFuzzyValue(tolerance)
        if cut1.IsDone():
            results['diff1'] = cut1.Shape()
        else:
            errors.append("diff1")
    except Exception as e:
        errors.append(f"diff1: {str(e)}")
    
    # 4. Разность (B \ A) — уникальные части модели 2
    try:
        cut2 = BRepAlgoAPI_Cut(solid2, solid1)
        cut2.SetFuzzyValue(tolerance)
        if cut2.IsDone():
            results['diff2'] = cut2.Shape()
        else:
            errors.append("diff2")
    except Exception as e:
        errors.append(f"diff2: {str(e)}")
    
    # 5. Симметрическая разность = (A\B) ∪ (B\A)
    if 'diff1' in results and 'diff2' in results:
        try:
            symdiff = BRepAlgoAPI_Fuse(results['diff1'], results['diff2'])
            symdiff.SetFuzzyValue(tolerance)
            if symdiff.IsDone():
                results['symdiff'] = symdiff.Shape()
        except:
            pass  # Не критично, если не удалось объединить различия
    
    # Добавляем информацию об ошибках
    results['errors'] = errors if errors else None
    
    return results

<>:2: DeprecationWarning: invalid escape sequence '\ '
<>:2: DeprecationWarning: invalid escape sequence '\ '
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/2176951558.py:2: DeprecationWarning: invalid escape sequence '\ '
  """


In [136]:
def compute_volumes(shape):
    """Вычисляет объём и площадь поверхности формы"""
    from OCC.Core.GProp import GProp_GProps
    from OCC.Core.BRepGProp import brepgprop_VolumeProperties, brepgprop_SurfaceProperties
    
    volume = 0.0
    area = 0.0
    
    try:
        props = GProp_GProps()
        brepgprop_VolumeProperties(shape, props)
        volume = props.Mass()
    except:
        pass  # Для поверхностей объём = 0
    
    try:
        props = GProp_GProps()
        brepgprop_SurfaceProperties(shape, props)
        area = props.Mass()
    except:
        pass
    
    return volume, area


def analyze_differences(shape1, shape2_aligned, tolerance=1e-5):
    """
    Полный анализ различий между моделями:
    - Объёмы уникальных частей
    - Процент различий относительно общей модели
    - Рекомендации по интерпретации
    """
    results = boolean_operations(shape1, shape2_aligned, tolerance)
    
    if results['errors']:
        print("⚠️ Предупреждения при булевых операциях:")
        for err in results['errors']:
            print(f"   • {err}")
    
    # Вычисляем объёмы
    vol1, area1 = compute_volumes(shape1)
    vol2, area2 = compute_volumes(shape2_aligned)
    
    vol_diff1 = compute_volumes(results['diff1'])[0] if 'diff1' in results else 0.0
    vol_diff2 = compute_volumes(results['diff2'])[0] if 'diff2' in results else 0.0
    vol_common = compute_volumes(results['common'])[0] if 'common' in results else 0.0
    
    # Процент различий
    total_vol = max(vol1, vol2)
    diff_percent = ((vol_diff1 + vol_diff2) / total_vol * 100) if total_vol > 0 else 0.0
    
    return {
        'volume_model1': vol1,
        'volume_model2': vol2,
        'volume_unique_to_1': vol_diff1,
        'volume_unique_to_2': vol_diff2,
        'volume_common': vol_common,
        'difference_percent': diff_percent,
        'results': results
    }


In [137]:
def boolean_operations(shape1, shape2_aligned, tolerance=1e-5):
    """
    Выполняет булевы операции для сравнения совмещённых моделей
    (полностью совместимо с pythonocc-core 7.9.0 без приведения типов)
    
    Возвращает словарь с результатами:
        - 'fuse': объединение (A ∪ B)
        - 'common': пересечение (A ∩ B)
        - 'diff1': разность (A \ B) — что есть в модели 1, но нет во 2
        - 'diff2': разность (B \ A) — что есть в модели 2, но нет в 1
        - 'symdiff': симметрическая разность — ВСЕ различия
    """
    from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse, BRepAlgoAPI_Common, BRepAlgoAPI_Cut
    
    results = {}
    errors = []
    
    # 1. Объединение (A ∪ B)
    try:
        fuse = BRepAlgoAPI_Fuse(shape1, shape2_aligned)
        fuse.SetFuzzyValue(tolerance)  # Улучшает стабильность для близких поверхностей
        if fuse.IsDone():
            results['fuse'] = fuse.Shape()
        else:
            errors.append("fuse: операция не выполнена")
    except Exception as e:
        errors.append(f"fuse: {str(e)}")
    
    # 2. Пересечение (A ∩ B)
    try:
        common = BRepAlgoAPI_Common(shape1, shape2_aligned)
        common.SetFuzzyValue(tolerance)
        if common.IsDone():
            results['common'] = common.Shape()
        else:
            errors.append("common: операция не выполнена")
    except Exception as e:
        errors.append(f"common: {str(e)}")
    
    # 3. Разность (A \ B) — уникальные части модели 1
    try:
        cut1 = BRepAlgoAPI_Cut(shape1, shape2_aligned)
        cut1.SetFuzzyValue(tolerance)
        if cut1.IsDone():
            results['diff1'] = cut1.Shape()
        else:
            errors.append("diff1: операция не выполнена")
    except Exception as e:
        errors.append(f"diff1: {str(e)}")
    
    # 4. Разность (B \ A) — уникальные части модели 2
    try:
        cut2 = BRepAlgoAPI_Cut(shape2_aligned, shape1)
        cut2.SetFuzzyValue(tolerance)
        if cut2.IsDone():
            results['diff2'] = cut2.Shape()
        else:
            errors.append("diff2: операция не выполнена")
    except Exception as e:
        errors.append(f"diff2: {str(e)}")
    
    # 5. Симметрическая разность = (A\B) ∪ (B\A)
    if 'diff1' in results and 'diff2' in results:
        try:
            symdiff = BRepAlgoAPI_Fuse(results['diff1'], results['diff2'])
            symdiff.SetFuzzyValue(tolerance)
            if symdiff.IsDone():
                results['symdiff'] = symdiff.Shape()
        except:
            pass  # Не критично, если не удалось объединить различия
    
    # Добавляем информацию об ошибках
    if errors:
        results['errors'] = errors
    
    return results


def compute_volumes(shape):
    """Вычисляет объём и площадь поверхности формы (работает с любым TopoDS_Shape)"""
    from OCC.Core.GProp import GProp_GProps
    from OCC.Core.BRepGProp import brepgprop_VolumeProperties, brepgprop_SurfaceProperties
    
    volume = 0.0
    area = 0.0
    
    # Объём (работает только для тел)
    try:
        props = GProp_GProps()
        brepgprop_VolumeProperties(shape, props)
        volume = props.Mass()
    except:
        volume = 0.0  # Для поверхностей объём = 0
    
    # Площадь поверхности (работает для всех типов)
    try:
        props = GProp_GProps()
        brepgprop_SurfaceProperties(shape, props)
        area = props.Mass()
    except:
        area = 0.0
    
    return volume, area


def analyze_differences(shape1, shape2_aligned, tolerance=1e-5):
    """
    Полный анализ различий между моделями через булевы операции
    """
    results = boolean_operations(shape1, shape2_aligned, tolerance)
    
    # Вычисляем объёмы
    vol1, area1 = compute_volumes(shape1)
    vol2, area2 = compute_volumes(shape2_aligned)
    
    vol_diff1 = compute_volumes(results['diff1'])[0] if 'diff1' in results else 0.0
    vol_diff2 = compute_volumes(results['diff2'])[0] if 'diff2' in results else 0.0
    vol_common = compute_volumes(results['common'])[0] if 'common' in results else 0.0
    
    # Процент различий
    total_vol = max(vol1, vol2)
    diff_percent = ((vol_diff1 + vol_diff2) / total_vol * 100) if total_vol > 0 else 0.0
    
    return {
        'volume_model1': vol1,
        'volume_model2': vol2,
        'volume_unique_to_1': vol_diff1,
        'volume_unique_to_2': vol_diff2,
        'volume_common': vol_common,
        'difference_percent': diff_percent,
        'area_model1': area1,
        'area_model2': area2,
        'results': results
    }


def save_boolean_results(results, output_dir="outputs/boolean"):
    """
    Сохраняет результаты булевых операций в STEP-файлы
    """
    from OCC.Core.STEPControl import STEPControl_Writer, STEPControl_AsIs
    import os
    
    os.makedirs(output_dir, exist_ok=True)
    saved = []
    
    mapping = {
        'fuse': 'union.step',
        'common': 'intersection.step',
        'diff1': 'unique_to_model1.step',
        'diff2': 'unique_to_model2.step',
        'symdiff': 'all_differences.step'
    }
    
    for key, filename in mapping.items():
        if key in results:
            try:
                writer = STEPControl_Writer()
                writer.Transfer(results[key], STEPControl_AsIs)
                status = writer.Write(os.path.join(output_dir, filename))
                if status == 1:  # IFSelect_RetDone
                    saved.append(filename)
            except Exception as e:
                print(f"⚠️ Не удалось сохранить {filename}: {e}")
    
    return saved

<>:2: DeprecationWarning: invalid escape sequence '\ '
<>:2: DeprecationWarning: invalid escape sequence '\ '
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/652969966.py:2: DeprecationWarning: invalid escape sequence '\ '
  """


In [138]:
# Предполагаем, что у вас уже есть:
#   shape1 — первая модель
#   shape2_aligned — вторая модель после совмещения

print("🔍 Анализ различий между моделями через булевы операции...")

# 1. Выполняем булевы операции
analysis = analyze_differences(shape1, shape2_aligned, tolerance=1e-4)

# 2. Выводим количественные результаты
print(f"\n📊 Количественный анализ:")
print(f"   Объём модели 1:        {analysis['volume_model1']:.2f} мм³")
print(f"   Объём модели 2:        {analysis['volume_model2']:.2f} мм³")
print(f"   Уникально модели 1:    {analysis['volume_unique_to_1']:.2f} мм³ ({analysis['volume_unique_to_1']/analysis['volume_model1']*100:.1f}%)")
print(f"   Уникально модели 2:    {analysis['volume_unique_to_2']:.2f} мм³ ({analysis['volume_unique_to_2']/analysis['volume_model2']*100:.1f}%)")
print(f"   Общая часть:           {analysis['volume_common']:.2f} мм³")
print(f"   📌 Различия:           {analysis['difference_percent']:.2f}% от общей модели")

# 3. Интерпретация результатов
if analysis['difference_percent'] < 0.1:
    print("\n✅ Модели практически идентичны (различия < 0.1%)")
elif analysis['difference_percent'] < 1.0:
    print("\n⚠️  Модели очень похожи (различия < 1%) — возможны допуски производства")
elif analysis['difference_percent'] < 5.0:
    print("\n🔶 Модели умеренно различаются (различия 1-5%) — проверьте функциональные элементы")
else:
    print("\n❌ Модели значительно различаются (различия > 5%) — требуется ручная проверка")

# 4. Сохраняем результаты для визуализации в FreeCAD
saved_files = save_boolean_results(analysis['results'], "outputs/boolean")

print(f"\n💾 Сохранены файлы для визуализации в FreeCAD:")
for fname in saved_files:
    print(f"   • outputs/boolean/{fname}")

print("\n💡 Советы по визуализации в FreeCAD:")
print("   1. Откройте 'all_differences.step' — это ВСЕ различия между моделями")
print("   2. Примените цвета:")
print("      • Красный: уникальные части модели 1 (unique_to_model1.step)")
print("      • Синий: уникальные части модели 2 (unique_to_model2.step)")
print("   3. Для анализа стыковки откройте 'intersection.step' — общая зона контакта")

🔍 Анализ различий между моделями через булевы операции...

📊 Количественный анализ:
   Объём модели 1:        68429.20 мм³
   Объём модели 2:        68429.20 мм³
   Уникально модели 1:    68429.20 мм³ (100.0%)
   Уникально модели 2:    68429.20 мм³ (100.0%)
   Общая часть:           0.00 мм³
   📌 Различия:           200.00% от общей модели

❌ Модели значительно различаются (различия > 5%) — требуется ручная проверка

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : outputs/boolean/union.step(1806 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******


/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/652969966.py:90: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(shape, props)
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/652969966.py:98: DeprecationWarning: Call to deprecated function brepgprop_SurfaceProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.SurfaceProperties
  brepgprop_SurfaceProperties(shape, props)
/var/folders/0t/6sq2wg4j0416j_4tk2z0wgfh0000gn/T/ipykernel_16267/652969966.py:90: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_Vo